In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator
import glob
import pandas
import matplotlib.pyplot as plt
import seaborn

In [ ]:
outdir = "../outputs/plots/"

In [ ]:
def extract_tb_tag(tb_path, tag="Loss/valid"):
    data = summary_iterator(tb_path)

    df = pandas.DataFrame()
    vals = []
    for e in data:
        for v in e.summary.value:
            if v.tag == tag:
                vals.append(v.simple_value)
    df[tag] = vals
    return df

In [ ]:
tb_logs = glob.glob("../training-outputs/240515_fullstats//**/events.out.tfevents.*", recursive=True)
tb_logs

In [ ]:
#get the training and model types 
tb_logs_names = [tuple(name.split("/")[3:5]) for name in tb_logs]
tb_logs_names

In [ ]:
tb_logs_dict = {k: v for (k, v) in zip(tb_logs_names, tb_logs)}

In [ ]:
loss_train = {k: extract_tb_tag(path, "Loss/train") for k, path in tb_logs_dict.items()} 
loss_valid = {k: extract_tb_tag(path, "Loss/valid") for k, path in tb_logs_dict.items()} 

In [ ]:
training_type = "jet_regression"
training_title = "$p_T$ regression"
ylim_train = 0.02
ylim_valid = 0.02

# training_type = "dm_multiclass"
# training_title = "Decay mode classification"
# ylim_train = 0.5
# ylim_valid = 0.05

In [ ]:
plt.plot(loss_train[(training_type, "ParticleTransformer")], label="ParticleTransformer")
plt.plot(loss_train[(training_type, "LorentzNet")], label="LorentzNet")
plt.plot(loss_train[(training_type, "SimpleDNN")], label="SimpleDNN")
plt.ylim(0.0, ylim_train)
plt.legend(loc="best")
plt.ylabel("Training loss")
plt.xlabel("Training epoch")
plt.title(training_title)
plt.savefig(outdir + "/train_loss_{}.pdf".format(training_type))

In [ ]:
plt.plot(loss_valid[("jet_regression", "ParticleTransformer")], label="ParticleTransformer")
plt.plot(loss_valid[("jet_regression", "LorentzNet")], label="LorentzNet")
plt.plot(loss_valid[("jet_regression", "SimpleDNN")], label="SimpleDNN")
plt.ylim(0.0, ylim_valid)
plt.legend(loc="best")
plt.ylabel("Validation loss")
plt.xlabel("Training epoch")
plt.savefig(outdir + "/valid_loss_{}.pdf".format(training_type))